In [2]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as c
import sympy as sy
import scipy.constants as pc
from sympy import *

In [3]:
# Funktionen
def am(werte):
    return np.mean(werte)

def std(werte):
    summen_der_quadrate = 0
    for wert in werte:
        summen_der_quadrate+=(wert-am(werte))**2       
    return np.sqrt(summen_der_quadrate/(werte.size-1))

def stu(werte):
    return std(werte)/np.sqrt(werte.size)

def fehler(f, werte, unsicherheiten, variablen):
    zMax = 0
    for i in range(werte.size):
        dif = np.abs(sy.diff(f, variablen[i]))
        zMax += (dif * unsicherheiten[i])**2
    zMax = sqrt(zMax)
    for j in range(werte.size):
        zMax = sub(zMax, variablen[j], werte[j])
    return zMax

def fehlerMax(f, werte, unsicherheiten, variablen):
    zMax = 0
    for i in range(werte.size):
        dif = np.abs(sy.diff(f, variablen[i]))
        zMax += dif * unsicherheiten[i]
    for j in range(werte.size):
        zMax = sub(zMax, variablen[j], werte[j])
    return zMax

def groessterBeitrag(f, werte, unsicherheiten, variablen):
    zMax = 0
    zMaxArray = np.array([])
    for i in range (werte.size):
        zMax = (np.abs(sy.diff(f, variablen[i])) * unsicherheiten[i])
        for j in range(werte.size):
            zMax = sub(zMax, variablen[j], werte[j])
        x = np.array([zMax])
        zMaxArray = np.append(zMaxArray, x)
    return variablen[np.argmax(zMaxArray)]

def sub(zMax, variable, wert):
    return zMax.subs([(variable, wert)])

def einfachGaussAdd(unsicherheiten, koeffizienten):
    dz = 0
    for i in range (unsicherheiten.size):
        dz += (unsicherheiten[i]*unsicherheiten[i])**2
    return sqrt(dz)

def einfachGaussMult(unsicherheiten, werte, exponenten):
    dz = 0
    for i in range (unsicherheiten.size):
        dz += (exponenten[i]*unsicherheiten[i]/werte[i])**2
    return sqrt(dz)

def vertraeglichkeit(am_1, am_2, stu_1, stu_2):
    return (np.abs(am_1-am_2)/(sqrt((stu_1**2)*(stu_2**2))))

def bestA(x, y):
    return (sum(x**2)*sum(y)-sum(x)*sum(x*y))/(x.size*sum(x**2)-sum(x)**2)

def bestB(x, y):
    return (x.size*sum(x*y)-sum(x)*sum(y))/(x.size*sum(x**2)-sum(x)**2)

def streuung(x, y):
    s = 0
    for i in range(x.size):
        s += (y[i]+(bestA(x, y)+bestB(x, y)*x[i]))**2
    return np.sqrt(s/(x.size-2))

def stuA(x, y):
    s = streuung(x, y)
    return s*np.sqrt(sum(x**2)/((x.size*sum(x**2)-sum(x)**2)))

def stuB(x, y):
    s = streuung(x, y)
    return s*np.sqrt(x.size/((x.size*sum(x**2)-sum(x)**2)))

In [12]:
#Werte 

b = np.array([5.0, 10.0, 15.0, 20.0, 22.5, 25.0])# mm (+- .5)
b_std = 0.5

r_1 = 4.4 # .05
r_1_std = 0.05

r_außen = 638.0/2.0 # +-1mm

s = np.array([[778.0, 787.0, 789.0, 797.0, 800.0, 801.0, 801.0, 802.0, 804.0, 805.0, 807.0, 818.0, 827.0, 840.0, 881.0],
              [578.0, 588.0, 646.0, 656.0, 661.0, 666.0, 667.0, 670.0, 670.0, 674.0, 710.0, 718.0, 726.0, 726.0, 749.0], 
              [510.0, 549.0, 548.0, 549.0, 550.0, 550.0, 550.0, 551.0, 552.0, 555.0, 556.0, 561.0, 563.0, 564.0, 576.0], 
              [332.0, 362.0, 367.0, 373.0, 377.0, 379.0, 381.0, 381.0, 382.0, 384.0, 398.0, 400.0, 401.0, 403.0, 407.0], 
              [113.0, 197.0, 198.0, 234.0, 238.0, 239.0, 251.0, 254.0, 258.0, 264.0, 267.0, 268.0, 280.0, 289.0, 333.0],
              [37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0]]) # mm (+- 1)

#s_25 = np.array([37.0])

#s_am = np.array([]) # mm (+- 1)
s_tot = r_außen*2.0*pc.pi # mm (+- 1)

#for el in s:
    #s_am = np.append(s_am, am(el))

theta = ((s*2.0*pc.pi)/s_tot) # rad
theta_am = np.array([]) # rad
theta_std = np.array([])

for el in theta:
    theta_am = np.append(theta_am, am(el))
    theta_std = np.append(theta_std, std(el))

#theta_am = np.append(theta_am, (s_25*2.0*pc.pi)/s_tot)
#theta_std = np.append(theta_std, 0)

cos_theta = np.cos(theta/2)

cos_theta_am =  np.array([])
cos_theta_std =  np.array([])

for el in cos_theta:
    cos_theta_am = np.append(cos_theta_am, am(el))
    cos_theta_std = np.append(cos_theta_std, std(el))

    
# Fehlerfortpflanzung:

print("b",b)
print("b_std", b_std)

print("Cos(theta/2)", cos_theta_am)
print("Cos(theta/2)_std", cos_theta_std)

print("r_1", r_1)
print("r_1_std", r_1_std)

def gaus_formel(b, cos_the, r_1, b_std, cos_the_std, r_1_std):
    return np.sqrt(((1/cos_the)*cos_the_std)**2 + ((-b/cos_the**2)*cos_the_std)**2 + ((-1)*r_1_std)**2)

#Versuchsreihen:

fehler_gesamt = np.zeros(6)

for i in range(6):
    fehler_gesamt[i] = gaus_formel(b[i], cos_theta_am[i], r_1, b_std, cos_theta_std[i], r_1_std )
    
print("Fehler von jeder messreihe", fehler_gesamt)
print("Fehler durchschnitt", am(fehler_gesamt))
print(am((b/cos_theta_am)-r_1))
    
    








b [ 5.  10.  15.  20.  22.5 25. ]
b_std 0.5
Cos(theta/2) [0.29776589 0.49111494 0.64801858 0.82586212 0.92419951 0.99831883]
Cos(theta/2)_std [3.79184474e-02 6.47727170e-02 1.65051069e-02 1.67561244e-02
 2.67371932e-02 2.29838002e-16]
r_1 4.4
r_1_std 0.05
Fehler von jeder messreihe [2.14268194 2.68920565 0.5922334  0.49430124 0.70667985 0.05      ]
Fehler durchschnitt 1.1125170132360254
17.917607017870107
